In [26]:
#basics
import random
import pandas as pd
import torch
#extra:
import os
import nltk
from lxml import etree

In [57]:
#sentence_id : id for the sentence, can be found in the xml data
#token_id: id for the word in the vocabulary
#char_start_id: where the character starts in the sentence, can be found in the xml data
#char_end_id: where the character ends in the sentence, can be found in the xml data
#split: which data split the token belong to, e.g. TRAIN, VAL or TEST.
#ner_id: id of the NER label, e.g. if we have 3 labels there would be 3 ids.from glob import glob

def parse_data(data_dir):     
    vocab = [] #keeping track of unique words in the data
    data_dir = glob("{}/*".format(data_dir)) #glob returns a possibly-empty list of path names that match data_dir 
                                            #...in this case a list with the two subdirectories 'Test' and 'Train'                                          
    for subdir in data_dir: #looping through 'Test' and 'Train'
        split = os.path.basename(subdir) #get the directory name without path
        subdir = glob("{}/*".format(subdir))
        for folder in subdir:  #looping through 'Test for DDI Extraction task' and 'Test for DrugNER task'
            folder = glob("{}/*".format(folder))
            for subfolder in folder: #looping through 'DrugBank' and 'MedLine'
                subfolder = glob("{}/*".format(subfolder))
                for xml_file in subfolder:
                    parse_xml(xml_file, split, vocab)
                    print("##################################################################")
                        
def parse_xml(xml_file, split, vocab):    
    tree = etree.parse(xml_file)
    root = tree.getroot()
    
    for elem in root: #loop over sentence tags
        if elem.tag == 'sentence':
            sent_id = elem.attrib['id'] #get sentence id
            #TODO: separera ord med bindestreck mha annan tokenizer än split :/ :/ :/
            text = elem.attrib['text']  #get the sentence as a string of text
            char_pos = -1 #variable for keeping track of character-based positions of the words in the sentence
            nltk_tokens = nltk.word_tokenize(text)
            for token in nltk_tokens:
            #for word in text.split():
                char_pos += 1
                char_start = char_pos
                char_end = char_start + len(token)-1
                token_id, vocab = map_token_to_id(token, vocab)
                data_instance = [sent_id, token, token_id, char_start, char_end, split]
                print(data_instance)
                char_pos=char_end+1 #increase by 1 to account for the whitespace between the current and the next word 
    #for subelem in elem:
    #if subelem.tag == 'entity'
        #print('elem.attrib[text]: ', text)
        #for subelem in elem:                
    #return data_instance, vocabulary
    
def map_token_to_id(token, vocab):
    vocab = vocab
    if token not in vocab:
        vocab.append(token)
    token_id = vocab.index(token)
    return token_id, vocab

#def get_train_val_test_tag()
    
    
# one specific item's data
#print('\nItem #2 data:')
#print(root[0][1].text)

In [58]:
parse_data('/home/guserbto@GU.GU.SE/lt2316-h20-aa/DDICorpus')

['DDI-DrugBank.d610.s0', 'Pharmacokinetic', 0, 0, 14, 'Test']
['DDI-DrugBank.d610.s0', 'properties', 1, 16, 25, 'Test']
['DDI-DrugBank.d610.s0', 'of', 2, 27, 28, 'Test']
['DDI-DrugBank.d610.s0', 'abacavir', 3, 30, 37, 'Test']
['DDI-DrugBank.d610.s0', 'were', 4, 39, 42, 'Test']
['DDI-DrugBank.d610.s0', 'not', 5, 44, 46, 'Test']
['DDI-DrugBank.d610.s0', 'altered', 6, 48, 54, 'Test']
['DDI-DrugBank.d610.s0', 'by', 7, 56, 57, 'Test']
['DDI-DrugBank.d610.s0', 'the', 8, 59, 61, 'Test']
['DDI-DrugBank.d610.s0', 'addition', 9, 63, 70, 'Test']
['DDI-DrugBank.d610.s0', 'of', 2, 72, 73, 'Test']
['DDI-DrugBank.d610.s0', 'either', 10, 75, 80, 'Test']
['DDI-DrugBank.d610.s0', 'lamivudine', 11, 82, 91, 'Test']
['DDI-DrugBank.d610.s0', 'or', 12, 93, 94, 'Test']
['DDI-DrugBank.d610.s0', 'zidovudine', 13, 96, 105, 'Test']
['DDI-DrugBank.d610.s0', 'or', 12, 107, 108, 'Test']
['DDI-DrugBank.d610.s0', 'the', 8, 110, 112, 'Test']
['DDI-DrugBank.d610.s0', 'combination', 14, 114, 124, 'Test']
['DDI-DrugBank.d

['DDI-DrugBank.d776.s26', 'controlled', 495, 2, 11, 'Test']
['DDI-DrugBank.d776.s26', 'study', 47, 13, 17, 'Test']
['DDI-DrugBank.d776.s26', 'found', 496, 19, 23, 'Test']
['DDI-DrugBank.d776.s26', 'that', 315, 25, 28, 'Test']
['DDI-DrugBank.d776.s26', 'concomitant', 25, 30, 40, 'Test']
['DDI-DrugBank.d776.s26', 'fluconazole', 430, 42, 52, 'Test']
['DDI-DrugBank.d776.s26', '200', 497, 54, 56, 'Test']
['DDI-DrugBank.d776.s26', 'mg', 56, 58, 59, 'Test']
['DDI-DrugBank.d776.s26', 'once', 498, 61, 64, 'Test']
['DDI-DrugBank.d776.s26', 'daily', 58, 66, 70, 'Test']
['DDI-DrugBank.d776.s26', 'and', 15, 72, 74, 'Test']
['DDI-DrugBank.d776.s26', 'cisapride', 493, 76, 84, 'Test']
['DDI-DrugBank.d776.s26', '20', 499, 86, 87, 'Test']
['DDI-DrugBank.d776.s26', 'mg', 56, 89, 90, 'Test']
['DDI-DrugBank.d776.s26', 'four', 500, 92, 95, 'Test']
['DDI-DrugBank.d776.s26', 'times', 188, 97, 101, 'Test']
['DDI-DrugBank.d776.s26', 'a', 46, 103, 103, 'Test']
['DDI-DrugBank.d776.s26', 'day', 484, 105, 107, 'Tes

['DDI-DrugBank.d597.s23', '13', 806, 26, 27, 'Test']
['DDI-DrugBank.d597.s23', 'rheumatoid', 807, 29, 38, 'Test']
['DDI-DrugBank.d597.s23', 'arthritis', 808, 40, 48, 'Test']
['DDI-DrugBank.d597.s23', '(', 54, 50, 50, 'Test']
['DDI-DrugBank.d597.s23', 'RA', 809, 52, 53, 'Test']
['DDI-DrugBank.d597.s23', ')', 59, 55, 55, 'Test']
['DDI-DrugBank.d597.s23', 'patients', 50, 57, 64, 'Test']
['DDI-DrugBank.d597.s23', 'evaluated', 810, 66, 74, 'Test']
['DDI-DrugBank.d597.s23', 'the', 8, 76, 78, 'Test']
['DDI-DrugBank.d597.s23', 'effects', 262, 80, 86, 'Test']
['DDI-DrugBank.d597.s23', 'of', 2, 88, 89, 'Test']
['DDI-DrugBank.d597.s23', 'multiple', 158, 91, 98, 'Test']
['DDI-DrugBank.d597.s23', 'doses', 180, 100, 104, 'Test']
['DDI-DrugBank.d597.s23', 'of', 2, 106, 107, 'Test']
['DDI-DrugBank.d597.s23', 'meloxicam', 731, 109, 117, 'Test']
['DDI-DrugBank.d597.s23', 'on', 31, 119, 120, 'Test']
['DDI-DrugBank.d597.s23', 'the', 8, 122, 124, 'Test']
['DDI-DrugBank.d597.s23', 'pharmacokinetics', 22, 12

['DDI-DrugBank.d779.s6', 'Read', 1159, 0, 3, 'Test']
['DDI-DrugBank.d779.s6', 'the', 8, 5, 7, 'Test']
['DDI-DrugBank.d779.s6', 'circular', 1160, 9, 16, 'Test']
['DDI-DrugBank.d779.s6', 'for', 193, 18, 20, 'Test']
['DDI-DrugBank.d779.s6', 'lithium', 795, 22, 28, 'Test']
['DDI-DrugBank.d779.s6', 'preparations', 1161, 30, 41, 'Test']
['DDI-DrugBank.d779.s6', '.', 16, 43, 43, 'Test']
['DDI-DrugBank.d779.s7', 'Several', 1162, 0, 6, 'Test']
['DDI-DrugBank.d779.s7', 'studies', 231, 8, 14, 'Test']
['DDI-DrugBank.d779.s7', 'demonstrate', 476, 16, 26, 'Test']
['DDI-DrugBank.d779.s7', 'a', 46, 28, 28, 'Test']
['DDI-DrugBank.d779.s7', 'decrease', 620, 30, 37, 'Test']
['DDI-DrugBank.d779.s7', 'in', 40, 39, 40, 'Test']
['DDI-DrugBank.d779.s7', 'the', 8, 42, 44, 'Test']
['DDI-DrugBank.d779.s7', 'bioavailability', 705, 46, 60, 'Test']
['DDI-DrugBank.d779.s7', 'of', 2, 62, 63, 'Test']
['DDI-DrugBank.d779.s7', 'methyldopa', 1141, 65, 74, 'Test']
['DDI-DrugBank.d779.s7', 'when', 143, 76, 79, 'Test']
['DD

['DDI-DrugBank.d706.s0', 'e.g.', 663, 32, 35, 'Test']
['DDI-DrugBank.d706.s0', ',', 67, 37, 37, 'Test']
['DDI-DrugBank.d706.s0', 'reserpine', 1424, 39, 47, 'Test']
['DDI-DrugBank.d706.s0', ')', 59, 49, 49, 'Test']
['DDI-DrugBank.d706.s0', 'may', 84, 51, 53, 'Test']
['DDI-DrugBank.d706.s0', 'have', 113, 55, 58, 'Test']
['DDI-DrugBank.d706.s0', 'an', 38, 60, 61, 'Test']
['DDI-DrugBank.d706.s0', 'additive', 713, 63, 70, 'Test']
['DDI-DrugBank.d706.s0', 'effect', 30, 72, 77, 'Test']
['DDI-DrugBank.d706.s0', 'when', 143, 79, 82, 'Test']
['DDI-DrugBank.d706.s0', 'given', 283, 84, 88, 'Test']
['DDI-DrugBank.d706.s0', 'with', 60, 90, 93, 'Test']
['DDI-DrugBank.d706.s0', 'beta-blocking', 1425, 95, 107, 'Test']
['DDI-DrugBank.d706.s0', 'agents', 397, 109, 114, 'Test']
['DDI-DrugBank.d706.s0', '.', 16, 116, 116, 'Test']
['DDI-DrugBank.d706.s1', 'Patients', 162, 0, 7, 'Test']
['DDI-DrugBank.d706.s1', 'treated', 279, 9, 15, 'Test']
['DDI-DrugBank.d706.s1', 'with', 60, 17, 20, 'Test']
['DDI-DrugBank

...

['DDI-MedLine.d178.s2', 'acute', 2842, 154, 158, 'Test']
['DDI-MedLine.d178.s2', 'coronary', 4343, 160, 167, 'Test']
['DDI-MedLine.d178.s2', 'syndrome', 4344, 169, 176, 'Test']
['DDI-MedLine.d178.s2', '.', 16, 178, 178, 'Test']
##################################################################
['DDI-MedLine.d171.s0', '[', 3253, 0, 0, 'Test']
['DDI-MedLine.d171.s0', 'Infections', 5738, 2, 11, 'Test']
['DDI-MedLine.d171.s0', 'with', 60, 13, 16, 'Test']
['DDI-MedLine.d171.s0', 'human', 189, 18, 22, 'Test']
['DDI-MedLine.d171.s0', 'immunodeficiency', 4822, 24, 39, 'Test']
['DDI-MedLine.d171.s0', 'viruses', 5739, 41, 47, 'Test']
['DDI-MedLine.d171.s0', '.', 16, 49, 49, 'Test']
['DDI-MedLine.d171.s0', 'Part', 5740, 51, 54, 'Test']
['DDI-MedLine.d171.s0', 'II', 2158, 56, 57, 'Test']
['DDI-MedLine.d171.s0', ':', 90, 59, 59, 'Test']
['DDI-MedLine.d171.s0', 'Antiretroviral', 5741, 61, 74, 'Test']
['DDI-MedLine.d171.s0', 'drugs', 504, 76, 80, 'Test']
['DDI-MedLine.d171.s0', ',', 67, 82, 82, 'Test

['DDI-MedLine.d222.s5', 'medication', 600, 63, 72, 'Test']
['DDI-MedLine.d222.s5', ',', 67, 74, 74, 'Test']
['DDI-MedLine.d222.s5', 'full', 3015, 76, 79, 'Test']
['DDI-MedLine.d222.s5', 'disclosure', 5895, 81, 90, 'Test']
['DDI-MedLine.d222.s5', 'of', 2, 92, 93, 'Test']
['DDI-MedLine.d222.s5', 'the', 8, 95, 97, 'Test']
['DDI-MedLine.d222.s5', 'risks', 1738, 99, 103, 'Test']
['DDI-MedLine.d222.s5', 'and', 15, 105, 107, 'Test']
['DDI-MedLine.d222.s5', 'benefits', 4624, 109, 116, 'Test']
['DDI-MedLine.d222.s5', 'should', 166, 118, 123, 'Test']
['DDI-MedLine.d222.s5', 'be', 85, 125, 126, 'Test']
['DDI-MedLine.d222.s5', 'discussed', 4718, 128, 136, 'Test']
['DDI-MedLine.d222.s5', 'with', 60, 138, 141, 'Test']
['DDI-MedLine.d222.s5', 'the', 8, 143, 145, 'Test']
['DDI-MedLine.d222.s5', 'patient', 157, 147, 153, 'Test']
['DDI-MedLine.d222.s5', '.', 16, 155, 155, 'Test']
['DDI-MedLine.d222.s6', 'The', 43, 0, 2, 'Test']
['DDI-MedLine.d222.s6', 'significant', 19, 4, 14, 'Test']
['DDI-MedLine.d222

['DDI-MedLine.d148.s5', 'Rev-erb', 5997, 45, 51, 'Test']
['DDI-MedLine.d148.s5', 'localization', 6020, 53, 64, 'Test']
['DDI-MedLine.d148.s5', 'to', 21, 66, 67, 'Test']
['DDI-MedLine.d148.s5', 'dendrites', 6012, 69, 77, 'Test']
['DDI-MedLine.d148.s5', 'and', 15, 79, 81, 'Test']
['DDI-MedLine.d148.s5', 'spines', 6021, 83, 88, 'Test']
['DDI-MedLine.d148.s5', ',', 67, 90, 90, 'Test']
['DDI-MedLine.d148.s5', 'a', 46, 92, 92, 'Test']
['DDI-MedLine.d148.s5', 'process', 5432, 94, 100, 'Test']
['DDI-MedLine.d148.s5', 'that', 315, 102, 105, 'Test']
['DDI-MedLine.d148.s5', 'is', 127, 107, 108, 'Test']
['DDI-MedLine.d148.s5', 'mediated', 2816, 110, 117, 'Test']
['DDI-MedLine.d148.s5', 'by', 7, 119, 120, 'Test']
['DDI-MedLine.d148.s5', 'AMPA', 6022, 122, 125, 'Test']
['DDI-MedLine.d148.s5', 'receptor', 3229, 127, 134, 'Test']
['DDI-MedLine.d148.s5', 'activation', 216, 136, 145, 'Test']
['DDI-MedLine.d148.s5', 'and', 15, 147, 149, 'Test']
['DDI-MedLine.d148.s5', 'requires', 3180, 151, 158, 'Test']


['DDI-MedLine.d170.s0', '.', 16, 132, 132, 'Test']
['DDI-MedLine.d170.s1', '-Aminobutyric', 6198, 0, 12, 'Test']
['DDI-MedLine.d170.s1', 'acid', 853, 14, 17, 'Test']
['DDI-MedLine.d170.s1', '(', 54, 19, 19, 'Test']
['DDI-MedLine.d170.s1', 'GABA', 6199, 21, 24, 'Test']
['DDI-MedLine.d170.s1', ')', 59, 26, 26, 'Test']
['DDI-MedLine.d170.s1', 'ergic', 6200, 28, 32, 'Test']
['DDI-MedLine.d170.s1', 'transmission', 5314, 34, 45, 'Test']
['DDI-MedLine.d170.s1', 'plays', 6201, 47, 51, 'Test']
['DDI-MedLine.d170.s1', 'an', 38, 53, 54, 'Test']
['DDI-MedLine.d170.s1', 'important', 1196, 56, 64, 'Test']
['DDI-MedLine.d170.s1', 'role', 3566, 66, 69, 'Test']
['DDI-MedLine.d170.s1', 'in', 40, 71, 72, 'Test']
['DDI-MedLine.d170.s1', 'the', 8, 74, 76, 'Test']
['DDI-MedLine.d170.s1', 'initiation', 1637, 78, 87, 'Test']
['DDI-MedLine.d170.s1', 'of', 2, 89, 90, 'Test']
['DDI-MedLine.d170.s1', 'epileptic', 6202, 92, 100, 'Test']
['DDI-MedLine.d170.s1', 'activity', 818, 102, 109, 'Test']
['DDI-MedLine.d170.

['DDI-MedLine.d197.s5', 'activation', 216, 17, 26, 'Test']
['DDI-MedLine.d197.s5', 'of', 2, 28, 29, 'Test']
['DDI-MedLine.d197.s5', 'the', 8, 31, 33, 'Test']
['DDI-MedLine.d197.s5', 'pregnane', 6304, 35, 42, 'Test']
['DDI-MedLine.d197.s5', 'X', 6305, 44, 44, 'Test']
['DDI-MedLine.d197.s5', 'receptor', 3229, 46, 53, 'Test']
['DDI-MedLine.d197.s5', '(', 54, 55, 55, 'Test']
['DDI-MedLine.d197.s5', 'PXR', 6306, 57, 59, 'Test']
['DDI-MedLine.d197.s5', ')', 59, 61, 61, 'Test']
['DDI-MedLine.d197.s5', 'is', 127, 63, 64, 'Test']
['DDI-MedLine.d197.s5', 'likely', 555, 66, 71, 'Test']
['DDI-MedLine.d197.s5', 'to', 21, 73, 74, 'Test']
['DDI-MedLine.d197.s5', 'enhance', 1017, 76, 82, 'Test']
['DDI-MedLine.d197.s5', 'CYP24', 6307, 84, 88, 'Test']
['DDI-MedLine.d197.s5', 'expression', 3425, 90, 99, 'Test']
['DDI-MedLine.d197.s5', 'and', 15, 101, 103, 'Test']
['DDI-MedLine.d197.s5', 'the', 8, 105, 107, 'Test']
['DDI-MedLine.d197.s5', 'catabolism', 6308, 109, 118, 'Test']
['DDI-MedLine.d197.s5', 'of',

In [51]:
xml = '<a xmlns="test"><b xmlns="test"/></a>'
root = etree.fromstring(xml)
etree.tostring(root)

b'<a xmlns="test"><b xmlns="test"/></a>'